In [1]:
# coming soon

In [31]:
import pandas as pd
import ipywidgets as widgets
import numpy as np
import sys
sys.path.append("../../")
import os
import glob
import cv2

import plotly.graph_objs as go
import plotly.express as px
from ipywidgets import HBox, VBox, Button

In [32]:
log_directory = os.path.join("..", "logs")
csv_files = glob.glob(os.path.join(log_directory, "*.csv"))
filenames = [os.path.split(csv_file)[-1] for csv_file in csv_files]
opts = [csv_tuple for csv_tuple in zip(filenames, csv_files)]

In [33]:

available_metrix = ['abs_error',
       'mse', 'avg', 'eucledian', 'bad1', 'bad2', 'bad4', 'bad8']

metrics_selector = widgets.Dropdown(
    options=[(m,m) for m in available_metrix],
    description='Metrics:',
    value="bad2"
)
scene_selector = widgets.Dropdown(
    options=[(m,m) for m in ["teddy", "cones"]],
    description='Scene:'
)

nonoccluded = widgets.Dropdown(
    options=[("yes", False), ("No", True)],
    description='Nonoccluded:'
)


VBox([metrics_selector, scene_selector, nonoccluded])

In [34]:
def read_image_binary(path):
    with open(path, "rb") as f:
        b = f.read()
        return b

selected_file = "D:\\gdrive\\python_projects\\FYP_FINAL\\benchmarking\\MiddEval\\custom_log\\all_benchmarking.csv"
df = pd.read_csv(selected_file)
df = df.drop_duplicates()


df["loaded_imgs"] = [read_image_binary(path) for path in df["image_filename"]]
"""selected_scene_df = df[
                        (df["scene"] == scene_selector.value) &
                      (df["are_occlusions_errors"] ==nonoccluded.value)
                      ]"""
selected_scene_df = df 


In [35]:
from ipywidgets import Image, Layout

img_widget = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))


In [36]:
h_n_w = np.array(selected_scene_df["kernel_size"].str.split("x").to_list())
selected_scene_df["h"] = pd.to_numeric(h_n_w[:, 0])
selected_scene_df["w"] = pd.to_numeric(h_n_w[:, 1])
selected_scene_df = selected_scene_df.sort_values(by=["h", "w"])
number_of_samples = selected_scene_df.shape[0]

In [37]:
def get_figure_widget(df, x_label,y_label, title, use_color=False, color_dim=None):
    if(not use_color):
        fig= go.FigureWidget(px.scatter(df, x=x_label,
            y=y_label, hover_data=["match", "gap", "egap", "kernel_size", "kernel_spec", "preprocessing_method", "scene"]))
    else:
        fig= go.FigureWidget(px.scatter(df, x=x_label,
            y=y_label, hover_data=["match", "gap", "egap", 
                                   "kernel_size", "kernel_spec", "preprocessing_method", "scene"], color = color_dim
                                       ))
 
    fig.update_xaxes(showspikes=True)
    fig.update_yaxes(showspikes=True)
    fig.layout.title = title
    fig.data[0].marker.opacity = np.full(fig.data[0].x.shape, 0.5)
    return fig

def get_hover_function(img_widget, selected_scene_df):
    
    def hover_fn(trace, points, state):
        if(len(points.point_inds)>0):
            ind = points.point_inds[0]
            img_widget.value = selected_scene_df["loaded_imgs"].iloc[ind]
    return hover_fn

def bind_hover_function(figs, img_widget, selected_scene_df):
    for fig in figs:
        for trace in fig.data:
            trace.on_hover(get_hover_function(img_widget, selected_scene_df))
        
def get_brush_function(master_n_slaves, selected_scene_df):        
    def brush_fn(trace, points, state):
        inds = np.array(points.point_inds)
        if inds.size:
            selected = np.full(selected_scene_df.shape[0], 0.05)
            selected[inds] = 1
            for fig in master_n_slaves:
                with fig.batch_update():
                    fig.data[0].marker.opacity = selected
    return brush_fn
                
def bind_brush_function(master_n_slaves, selected_scene_df):
    func = get_brush_function(master_n_slaves, selected_scene_df)
    for fig in master_n_slaves:
        for trace in fig.data:
            trace.on_selection(func)
        
def get_reset_brush_function(master_n_slaves):
    def reset_fn(btn):
        selected = np.full(master_n_slaves[0].data[0].marker.opacity.shape, 0.05)
        for fig in master_n_slaves:
            with fig.batch_update():
                fig.data[0].marker.opacity = selected
    return reset_fn

def get_reset_brush_button(master_n_slaves, description_passed = "clear selection"):
    btn = Button(description=description_passed)
    reset_fn = get_reset_brush_function(master_n_slaves)
    btn.on_click(reset_fn)
    return btn

def get_dropdown_function(df, column, widget, figure_widget):     
        
    def validate():
        return True
        """        if widget.value in df[column].unique():
            return True
        else:
            return False"""


    def response(change):
        if validate():
            temp_df = df[df[column]==widget.value]
            x = temp_df['kernel_size']
            y = temp_df['match']
            with figure_widget.batch_update():
                figure_widget.data[0].x = x
                figure_widget.data[0].y = y

                print("I should have been updated.")
    return response

def bind_dropdown_function(df, column, dropdown_widget, figure_widget):
        dropdown_fn = get_dropdown_function(df, column, dropdown_widget, figure_widget)
        dropdown_widget.observe(dropdown_fn, names="value")

def get_dropdown_widget(df, column):
    dropdown_widget = widgets.Dropdown(
    description=column,
    options=df[column].unique()
    )
    return dropdown_widget        

In [38]:
fig_a = get_figure_widget (selected_scene_df, "experiment_id", metrics_selector.value, 
                           "Scene w.r.t."+metrics_selector.value)
fig_b = get_figure_widget (selected_scene_df, "match", "kernel_size", "Kernel sizes w.r.t. match values")



figs = [fig_a, fig_b]
bind_hover_function(figs, img_widget, selected_scene_df)
bind_brush_function(figs, selected_scene_df)

In [39]:

button = get_reset_brush_button(figs)


In [40]:
dashboard = VBox([button, fig_a,
                  HBox([img_widget, fig_b])])
fig_a.data[0].marker.opacity = np.full(selected_scene_df.shape[0], 1)

In [41]:
dashboard

    'data': [{'custo…

In [42]:
selected_scene_df["h"].unique()

array([ 1,  3,  5,  7,  9, 11, 13, 15], dtype=int64)

In [43]:
def get_hover_function2(img_widget, selected_scene_df):
    
    def hover_fn(trace, points, state):
        #print(points)
        if(len(points.point_inds)>0):
            ind = points.point_inds[0]
            img_widget.value = dfs[points.trace_index]["loaded_imgs"].iloc[ind]
    return hover_fn

def bind_hover_function2(figs, img_widget, selected_scene_df):
    for fig in figs:
        for trace in fig.data:
            trace.on_hover(get_hover_function2(img_widget, selected_scene_df))

In [44]:
def get_figure_widget_traced(df, x_label,y_label, trace_dim):
    trace_keys = df[trace_dim].unique()
    fig = go.Figure()
    dfs=[]

    for k in trace_keys:
        temp_df = df[df[trace_dim]==k]
        dfs.append(temp_df)

        label = ["Scene: {0}<br>kernel size: {1}<br>match:{2}<br>bad2: {3}".format(a,b,c,d) 
                 for a,b,c,d in zip(temp_df["scene"],temp_df["kernel_size"],temp_df["match"], temp_df["bad2"])
                ]
        
        
        fig.add_trace(go.Scatter(x=temp_df[x_label], y=temp_df[y_label], name=k,  hovertemplate=
        '%{text}', text = label, showlegend=True))

    fig.update_xaxes(showspikes=True)
    fig.update_yaxes(showspikes=True)
    return fig, dfs

In [45]:
"""fig, dfs = get_figure_widget_traced(df, "experiment_id", "bad2", "scene")
fig2 = get_figure_widget(df, "experiment_id", "kernel_size", "hello")
img_widget2 = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))
fig_widget = go.FigureWidget(fig)

bind_hover_function2([fig_widget], img_widget2, dfs)
VBox([fig_widget, HBox([img_widget2,fig2])])"""

'fig, dfs = get_figure_widget_traced(df, "experiment_id", "bad2", "scene")\nfig2 = get_figure_widget(df, "experiment_id", "kernel_size", "hello")\nimg_widget2 = Image(value=df["loaded_imgs"].iloc[0], \n                   layout=Layout(height=\'375px\', width=\'450px\'))\nfig_widget = go.FigureWidget(fig)\n\nbind_hover_function2([fig_widget], img_widget2, dfs)\nVBox([fig_widget, HBox([img_widget2,fig2])])'

In [46]:
fig, dfs = get_figure_widget_traced(df, "scene", "bad4", "experiment_id")
fig2 = get_figure_widget(df, "experiment_id", "kernel_size", "hello")
img_widget2 = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))
fig_widget = go.FigureWidget(fig)

bind_hover_function2([fig_widget], img_widget2, dfs)
VBox([fig_widget, HBox([img_widget2,fig2])])

    'data': [{'hovertemplate': '%{text}',
              'name': 'bm_20_1x1',
   …

In [1]:
%ls

 Volume in drive D is SharedData
 Volume Serial Number is 8653-19E2

 Directory of D:\gdrive\python_projects\FYP_FINAL\experiments\notebooks

03/07/2020  02:24 PM    <DIR>          .
03/07/2020  02:24 PM    <DIR>          ..
03/07/2020  02:02 PM    <DIR>          .ipynb_checkpoints
24/06/2020  11:51 PM           746,347 ALG_001-EXP_001-Baseline.ipynb
20/06/2020  05:30 AM           471,744 ALG_001-EXP_002-Baseline_H_res.ipynb
15/06/2020  04:24 PM         1,367,046 ALG_002_EXP_001-Baseline-MacLean_et_al.ipynb
15/06/2020  04:24 PM         1,135,155 ALG_002_EXP_002-Baseline-MacLean_et_al.ipynb
01/07/2020  03:44 AM           478,033 ALG_002_EXP_002-Baseline-MacLean_et_al_H_res.ipynb
15/06/2020  04:24 PM         1,285,333 ALG_003_EXP_001-Baseline-MacLean_et_al-Parallel_version.ipynb
25/06/2020  05:35 PM           494,304 ALG_004_EXP_001-Baseline-MacLean_et_al-Numba.ipynb
25/06/2020  01:15 AM         3,279,023 ALG_004_EXP_002-Baseline-MacLean_et_al-param_search.ipynb
25/06/2020  05:41 PM     